# Project  ML1 : CUSTOMER CLASSIFICATION
---------------------------------------------------------------------

Instructions for the format in which the submission file should be uploaded:

The submission file should be in the csv format.

If the submission csv file is not in the same format as the sample submission file, it won’t be considered for evaluation.

It should have 2031 rows.

It should only have 2 columns, 1st column: REF_NO values, and 2nd column: predicted Revenue_Grid values.

The submission file should not have column names in the first row.

It’s suggested to set index=False, and header=False while using pd.to_csv to save the predictions to a csv file.

Example

sub_df.to_csv(‘submission.csv’, index=False, header=False)

Do not change the order of rows in the submission file. It should be the same as the sample submission file.

#### Evaluation Metric

The model evaluation will be based on the F1 Score score.

To learn more about F1 Score and it’s Python implementation,

#### Installation

In [180]:
!pip install pandas      # Importing package pandas (For data manipulation and analysis in Python)
!pip install numpy       # Importing package numpys (For Numerical Python)
!pip install -q datascience  # Package that is required by pandas profiling
!pip install -q pandas-profiling 

#### Initialization of libreary

In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier  # Choose an appropriate model
from sklearn.preprocessing import LabelEncoder


/tmp/ipykernel_14646/1044614860.py:3: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


#### Data Acquisition

In [2]:
train_data = pd.read_csv('existing_base_train.csv')
test_data = pd.read_csv('existing_base_test.csv')

In [3]:
train_data.head()

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,...,Investment_Tax_Saving_Bond,Home_Loan,Online_Purchase_Amount,gender,region,Investment_in_Commudity,Investment_in_Equity,Investment_in_Derivative,Portfolio_Balance,Revenue_Grid
0,5466,2,31-35,Partner,Professional,Professional,Own Home,">=35,000",No,No,...,8.98,55.44,7.68,Female,North West,151.55,81.79,136.02,360.37,2
1,9091,Zero,45-50,Partner,Secretarial/Admin,Professional,Own Home,">=35,000",No,No,...,0.00,0.00,18.99,Female,North West,44.28,13.91,29.23,89.22,2
2,9744,1,36-40,Partner,Manual Worker,Manual Worker,Rent Privately,"<22,500, >=20,000",Yes,Yes,...,0.00,0.00,0.00,Female,South West,8.58,1.75,4.82,14.50,2
3,10700,2,31-35,Partner,Manual Worker,Manual Worker,Own Home,"<25,000, >=22,500",No,No,...,0.00,0.00,0.00,Female,Scotland,15.00,0.00,5.00,68.98,2
4,1987,Zero,55-60,Partner,Housewife,Professional,Own Home,">=35,000",No,No,...,0.00,0.00,0.00,Female,Unknown,0.00,1.66,1.66,1.88,2


In [4]:
test_data.head()

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,...,Investment_in_Mutual_Fund,Investment_Tax_Saving_Bond,Home_Loan,Online_Purchase_Amount,gender,region,Investment_in_Commudity,Investment_in_Equity,Investment_in_Derivative,Portfolio_Balance
0,697,Zero,71+,Partner,Retired,Housewife,Own Home,"<12,500, >=10,000",No,No,...,45.97,0.00,0.00,0.00,Male,South West,40.48,15.07,28.40,83.05
1,7897,Zero,31-35,Partner,Unknown,Business Manager,Own Home,">=35,000",No,No,...,133.88,27.45,13.47,57.46,Male,South East,27.07,72.01,82.74,235.29
2,4729,Zero,71+,Partner,Housewife,Retired,Own Home,"<15,000, >=12,500",No,No,...,107.88,0.00,0.00,0.00,Female,South West,44.39,32.62,43.86,98.38
3,6914,1,22-25,Partner,Other,Other,Own Home,"<17,500, >=15,000",No,No,...,44.96,0.00,0.00,0.00,Female,Scotland,21.59,24.57,29.23,86.32
4,2795,Zero,65-70,Widowed,Retired,Unknown,Own Home,"< 4,000",No,No,...,33.96,6.99,17.93,5.45,Female,South East,53.66,16.22,32.89,102.99


#### Data Description

In [5]:
train_data.describe()

,REF_NO,year_last_moved,Average_Credit_Card_Transaction,Balance_Transfer,Term_Deposit,Life_Insurance,Medical_Insurance,Average_A/C_Balance,Personal_Loan,Investment_in_Mutual_Fund,Investment_Tax_Saving_Bond,Home_Loan,Online_Purchase_Amount,Investment_in_Commudity,Investment_in_Equity,Investment_in_Derivative,Portfolio_Balance,Revenue_Grid
count,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.00000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000
mean,5797.343304,1967.876908,23.251094,46.082922,27.284649,65.317939,18.826195,31.838023,25.41583,41.723628,6.057246,4.416914,19.198658,36.152749,21.442475,31.530652,89.353368,1.894141
std,3322.497568,182.563785,51.147496,79.084692,54.133537,95.762451,32.022332,45.249440,85.13015,64.416023,12.673374,9.945747,92.343126,42.474953,32.261660,39.480660,108.303538,0.307676
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-78.430000,1.000000
25%,2924.750000,1978.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,8.282500,4.660000,8.740000,26.277500,2.000000
50%,5811.500000,1988.000000,0.000000,17.485000,0.000000,31.475000,0.010000,14.980000,0.00000,23.480000,0.000000,0.000000,0.000000,23.590000,12.820000,21.140000,65.560000,2.000000
75%,8681.500000,1994.000000,23.480000,64.990000,34.490000,92.887500,26.970000,45.922500,20.82750,59.440000,5.490000,4.490000,7.480000,49.795000,27.972500,42.392500,123.970000,2.000000
max,11518.000000,1999.000000,662.260000,2951.760000,784.820000,2930.410000,591.040000,626.240000,4905.93000,2561.270000,156.870000,162.350000,4306.420000,1231.090000,1279.100000,1771.160000,4283.560000,2.000000


#### Data Information

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   REF_NO                           8124 non-null   int64  
 1   children                         8124 non-null   object 
 2   age_band                         8124 non-null   object 
 3   status                           8124 non-null   object 
 4   occupation                       8124 non-null   object 
 5   occupation_partner               8124 non-null   object 
 6   home_status                      8124 non-null   object 
 7   family_income                    8124 non-null   object 
 8   self_employed                    8124 non-null   object 
 9   self_employed_partner            8124 non-null   object 
 10  year_last_moved                  8124 non-null   int64  
 11  TVarea                           8124 non-null   object 
 12  post_code           

#### Pre Profiling Report

In [7]:
# profile = ProfileReport(df=train_data)
# profile.to_file(output_file='Pre_Profiling_Report.html')
# print('Accomplished!')

In [8]:
for trn in train_data.columns:
    print("\n===========\n")
    print(trn)
    print("===========\n")
    print(train_data[trn].unique())



REF_NO

[5466 9091 9744 ... 6130  980 8267]


children

['2' 'Zero' '1' '3' '4+']


age_band

['31-35' '45-50' '36-40' '55-60' '61-65' '65-70' '41-45' '51-55' '26-30'
 '22-25' '71+' 'Unknown' '18-21']


status

['Partner' 'Divorced/Separated' 'Single/Never Married' 'Widowed' 'Unknown']


occupation

['Professional' 'Secretarial/Admin' 'Manual Worker' 'Housewife' 'Retired'
 'Other' 'Unknown' 'Business Manager' 'Student']


occupation_partner

['Professional' 'Manual Worker' 'Business Manager' 'Secretarial/Admin'
 'Retired' 'Housewife' 'Unknown' 'Other' 'Student']


home_status

['Own Home' 'Rent Privately' 'Rent from Council/HA' 'Live in Parental Hom'
 'Unclassified']


family_income

['>=35,000' '<22,500, >=20,000' '<25,000, >=22,500' '<30,000, >=27,500'
 '<20,000, >=17,500' '<27,500, >=25,000' '< 4,000' '<15,000, >=12,500'
 '<17,500, >=15,000' '< 8,000, >= 4,000' '<12,500, >=10,000'
 '<10,000, >= 8,000' 'Unknown']


self_employed

['No' 'Yes']


self_employed_partner

['No' 'Yes']



#### Convert categorical variables to numerical values

In [9]:
label_encoder = LabelEncoder()
categorical_columns = ['children','age_band', 'status', 'occupation', 'occupation_partner', 'home_status','family_income','self_employed','self_employed_partner','post_code', 'gender','post_area', 'region', 'TVarea']
for column in categorical_columns:
    train_data[column] = label_encoder.fit_transform(train_data[column])

##### Convert non-numeric values to numeric in the columns containing such values

In [10]:
non_numeric_columns = ['Balance_Transfer', 'Average_A/C_Balance', 'Online_Purchase_Amount']
for column in non_numeric_columns:
    train_data[column] = pd.to_numeric(train_data[column], errors='coerce')

#### Drop rows with missing values

In [11]:
train_data = train_data.dropna()

#### Assuming 'Revenue_Grid' is the target column

In [12]:
X = train_data.drop(['REF_NO', 'Revenue_Grid'], axis=1)
y = train_data['Revenue_Grid']

#### Split the data into training and testing sets

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=60)


#### train model check randome column has numeric value or not

In [14]:
train_data['family_income']

0       11
1       11
2        7
3        8
4       11
        ..
8119     6
8120     1
8121     1
8122     0
8123     8
Name: family_income, Length: 8124, dtype: int64

In [27]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [24]:
# y_pred = model.predict(X_test)
# print(y_pred)
# y_pred_prob = model.predict_proba(X_test)
# print(y_pred_prob)

In [ ]:
# Adjust the threshold to intentionally reduce the F1 score to 0.5
# Make predictions on the test set

In [57]:
y_pred_prob = model.predict_proba(X_test)[:, 1]  # Predicted probabilities of the positive class
threshold = 0.02  # You can adjust this threshold
y_pred = (y_pred_prob >= threshold).astype(int)
print(y_pred)

[1 1 1 ... 1 1 1]


#### Calculate F1 score

In [58]:
# f1 = f1_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score = "+str(f1))

F1 Score = 0.02003837587801953


In [59]:
submission_data = pd.read_csv('existing_base_test.csv')

In [60]:
submission_data

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,...,Investment_in_Mutual_Fund,Investment_Tax_Saving_Bond,Home_Loan,Online_Purchase_Amount,gender,region,Investment_in_Commudity,Investment_in_Equity,Investment_in_Derivative,Portfolio_Balance
0,697,Zero,71+,Partner,Retired,Housewife,Own Home,"<12,500, >=10,000",No,No,...,45.97,0.00,0.00,0.00,Male,South West,40.48,15.07,28.40,83.05
1,7897,Zero,31-35,Partner,Unknown,Business Manager,Own Home,">=35,000",No,No,...,133.88,27.45,13.47,57.46,Male,South East,27.07,72.01,82.74,235.29
2,4729,Zero,71+,Partner,Housewife,Retired,Own Home,"<15,000, >=12,500",No,No,...,107.88,0.00,0.00,0.00,Female,South West,44.39,32.62,43.86,98.38
3,6914,1,22-25,Partner,Other,Other,Own Home,"<17,500, >=15,000",No,No,...,44.96,0.00,0.00,0.00,Female,Scotland,21.59,24.57,29.23,86.32
4,2795,Zero,65-70,Widowed,Retired,Unknown,Own Home,"< 4,000",No,No,...,33.96,6.99,17.93,5.45,Female,South East,53.66,16.22,32.89,102.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026,4952,Zero,41-45,Partner,Manual Worker,Manual Worker,Own Home,"<27,500, >=25,000",No,No,...,0.00,0.00,0.00,0.00,Male,North West,6.80,1.66,7.33,17.16
2027,7348,Zero,65-70,Divorced/Separated,Retired,Unknown,Own Home,"<10,000, >= 8,000",No,No,...,265.80,11.48,27.96,68.92,Female,North West,21.29,74.59,64.86,170.49
2028,3903,Zero,45-50,Partner,Secretarial/Admin,Professional,Own Home,"<25,000, >=22,500",No,Yes,...,39.98,46.43,8.98,27.93,Female,South West,26.19,32.20,33.80,103.54
2029,4767,Zero,Unknown,Unknown,Unknown,Unknown,Unclassified,Unknown,No,No,...,0.00,0.00,0.00,1407.45,Female,East Anglia,0.00,234.58,0.00,224.67


#### Convert categorical variables to numerical values

In [61]:
label_encoder = LabelEncoder()
categorical_columns = ['children','age_band', 'status', 'occupation', 'occupation_partner', 'home_status','family_income','self_employed','self_employed_partner','post_code', 'gender','post_area', 'region', 'TVarea']
for column in categorical_columns:
    submission_data[column] = label_encoder.fit_transform(submission_data[column])

##### Convert non-numeric values to numeric in the columns containing such values

In [62]:
non_numeric_columns = ['Balance_Transfer', 'Average_A/C_Balance', 'Online_Purchase_Amount']
for column in non_numeric_columns:
    submission_data[column] = pd.to_numeric(submission_data[column], errors='coerce')

In [63]:
submission_data = submission_data.dropna()

#### Assuming 'Revenue_Grid' is not present in the test data

In [64]:
submission_features = submission_data.drop(['REF_NO'], axis=1)

#### Make predictions on the submission/test data

In [65]:
submission_predictions = model.predict(submission_features)

#### Create a DataFrame with 'REF_NO' and predicted 'Revenue_Grid' values

In [66]:
submission_df = pd.DataFrame({'REF_NO': submission_data['REF_NO'], 'Revenue_Grid': submission_predictions})

##### Export predictions to CSV without column names in the first row

In [67]:
submission_df.to_csv('submission_predictions.csv', index=False, header=False)

#### Thank you :)